<a href="https://colab.research.google.com/github/viraatdas/Purdue_CompVis_Workshop/blob/5_preprocessor(3)-shuffle-image/speed_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
print("wus good")

wus good


Import video files

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Import libraries

In [0]:
import os
import csv
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Organie file paths

In [0]:
train_video = "./drive/My Drive/speed_predict/train.mp4"
labels = "./drive/My Drive/speed_predict/train.txt"
output = "./drive/My Drive/speed_predict/data_preprocessed"

if not os.path.exists(output):
  os.mkdir(output)
  print(f"Directory {output} created")

Preprocessor class

In [0]:
# Input for the Optical Flow Farneback Method
flow_mat = None
image_scale = 0.5
nb_images = 1
win_size = 15
nb_iterations = 2
deg_expansion = 5
STD = 1.3

In [0]:
class PreProcessor:
  def __str__(self):
    # Indicate class description
    return "Pre-processor class"
  
  def grayscale(self, frame):
    # Converts frame to gray scale
    return cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
  
  def plot_training_speed(self, data):
    # plot labels
    data = np.loadtxt(data)
    plt.plot(data)
    plt.show()
  def generate_images(self, file_path, label_path, gray=False):
    # Takes the video file and generates images for each 
    # of it's frames. The time for each frame and the image path
    # for the corresponding frame is stored in a csv

    # Load the labels
    label = np.loadtxt(label_path)

    # Video file
    video_capture = cv2.VideoCapture(file_path)

    # Check if number of label is same as number of frames
    if len(labels) == video_capture.get(cv2.CAP_PROP_FRAME_COUNT):
      print("Labels are equal to number of frames")
    else:
      print("Chekc if labels are equal to number of frames")
      return

    with open("processed.csv", "w") as file:
      writer = csv.writer(file)
      time_between_frames = 1 / FRAME_RATE
      time_elapsed = 0

      for idx, i in enumerate(labels):
        ret, frame = video_capture.read()
        time_elapsed += time_between_frames
        if gray:
          frame = self.grayscale(frame)
        if ret:
          cv2.imshow("frame", frame)
          image_path = f"{output_path}_{str(time_elapsed)}.jpg"
          cv2.imwrite(image_path, frame)
          writer.writerow([image_path, time_elapsed, labels[idx]])
        if cv2.waitKey(1) & 0xFF == ord('q'):
          break
      video_capture.release()
      cv2.destroyAllWindows()

  def shuffle_frame_pairs(self, dataframe, val_split=1):
    # Shuffles pairs of frames from the video 
    # We need pair of frames for the optical flow
    # returns training and validation dataframe

    training_data = pd.DataFrame()
    validation_data = pd.DataFrame()
    df_len = dataframe.shape[0]

    for i in range(df_len - 1):
      idx1 = np.random.randint(df_len - 1)
      idx2 = idx1 + 1
      row1 = dataframe.iloc[[idx1]].reset_index()
      row2 = dataframe.iloc[[idx2]].reset_index()
      randInt = np.random.randint(9)

      if 0 <= randInt <= val_split:
        valid_frame = [validation_data, row1, row2]
        validation_data = pd.concat(
            valid_frames, axis=0, join='outer', ignore_index=False)
        if randInt >= val_split + 1:
          train_frame = [training_data, row1, row2]
          training_data = pd.concat(
              train_frames, axis=0, join='outer', ignore_index=False)
    return training_data, validation_data

